PDF Text extraction

In [2]:
import fitz
import geocoder
import pymysql
import fitz
from openai import OpenAI
from settings import *
import gradio as gr

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()

pdf_text = extract_text_from_pdf("cbc-report-format.pdf")
print(pdf_text[:50])


HAEMATOLOGY
COMPLETE BLOOD COUNT (CBC)
TEST
VALUE



Quadrant code ##NOT USED

In [20]:
# from qdrant_client import models, QdrantClient
# from sentence_transformers import SentenceTransformer

# # Initialize the encoder and Qdrant client
# encoder = SentenceTransformer("all-MiniLM-L6-v2")
# client = QdrantClient(":memory:")  # For in-memory use; replace with cluster host info for persistent storage


# collection_name = "medical_reports"
# client.create_collection(
#     collection_name=collection_name,
#     vectors_config=models.VectorParams(
#         size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the encoder
#         distance=models.Distance.COSINE,  # Use cosine similarity
#     ),
# )

# def chunk_text(text, chunk_size=500):
#     return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

# chunks = chunk_text(pdf_text)

# client.upload_points(
#     collection_name=collection_name,
#     points=[
#         models.PointStruct(
#             id=idx,
#             vector=encoder.encode(chunk).tolist(),
#             payload={"chunk": chunk}
#         )
#         for idx, chunk in enumerate(chunks)
#     ],
# )

# print(f"Stored {len(chunks)} chunks in Qdrant.")


# def query_relevant_chunks(query_text, top_k=5):
#     query_vector = encoder.encode(query_text).tolist()
#     hits = client.query_points(
#         collection_name=collection_name,
#         query=query_vector,
#         limit=top_k,
#     ).points
#     return [hit.payload["chunk"] for hit in hits]

# query = "summarize the medical findings"
# relevant_chunks = query_relevant_chunks(query)

# print("Relevant chunks for summarization:")
# for chunk in relevant_chunks:
#     print(chunk[:200], "...")



In [21]:
# from openai import OpenAI
# from settings import *
# import os

# generation_model_name: str
# temperature: float = 0.9
# top_p = 0.9
# max_tokens: int = 2048
# stream: bool = True
# llm_name: str = "Meta-Llama"

# monster_client = OpenAI(
#     base_url="https://llm.monsterapi.ai/v1/",
#     api_key=str(MONSTER_API_KEY)
# )

# monster_ai_model_name = {
#     "Google-Gemma": "google/gemma-2-9b-it",
#     "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
#     "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
#     "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
# }

# # Context and Summarization Prompt
# message = [
#     {"role": "system", "content": "You are an AI assistant skilled in summarizing complex medical reports."},
#     {"role": "user", "content": f"Summarize the following medical report in simple terms. Focus on the key findings, diagnoses, and recommendations. Report text:\n\n{pdf_text}"}
# ]

# # Generate Summary
# response = monster_client.chat.completions.create(
#     model=monster_ai_model_name[llm_name],
#     messages=message,
#     temperature=temperature,
#     top_p=top_p,
#     max_tokens=max_tokens,
#     stream=stream
# )

# # Collect Generated Text
# generated_text = ""
# for chunk in response:
#     if chunk.choices[0].delta.content is not None:
#         generated_text += chunk.choices[0].delta.content

# print("Generated Summary:")
# print(generated_text)



In [15]:

# PDF Text Extraction Function
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document[page_num]
        text += page.get_text()
    document.close()
    return text.strip()

# Monster API Client Configuration
generation_model_name: str
temperature: float = 0.9
top_p = 0.9
max_tokens: int = 2048
stream: bool = False  # Set to False for simplicity in Gradio
llm_name: str = "Meta-Llama"

monster_client = OpenAI(
    base_url="https://llm.monsterapi.ai/v1/",
    api_key=str(MONSTER_API_KEY)
)

monster_ai_model_name = {
    "Google-Gemma": "google/gemma-2-9b-it",
    "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
    "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
}

# Summarization Function
def summarize_pdf(file):
    try:
        # Extract text from the uploaded PDF
        pdf_text = extract_text_from_pdf(file.name)

        # Create context for the AI summarization
        message = [
            {"role": "system", "content": "You are an AI assistant skilled in summarizing complex medical reports."},
            {"role": "user", "content": f"Summarize the following medical report in simple terms. Focus on the key findings, diagnoses, and recommendations. Report text:\n\n{pdf_text}"}
        ]

        # Generate summary using Monster API
        response = monster_client.chat.completions.create(
            model=monster_ai_model_name[llm_name],
            messages=message,
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
            stream=False  # Simplify response handling
        )

        # Extract the summary from the response
        generated_text = response.choices[0].message.content
        return generated_text

    except Exception as e:
        return f"Error occurred: {e}"


# Gradio Interface
interface = gr.Interface(
    fn=summarize_pdf,
    inputs=gr.File(label="Upload your Medical Report (PDF)"),
    outputs=gr.Textbox(label="Generated Summary"),
    title="Medical Report Summarizer",
    description="Upload a medical report in PDF format, and this tool will generate a summary focusing on key findings, diagnoses, and recommendations."
)

# Launch the Interface
interface.launch()

response = monster_client.chat.completions.create(
    model=monster_ai_model_name[llm_name],
    messages=message,
    temperature=temperature,
    top_p=top_p,
    max_tokens=max_tokens,
    stream=stream
)

print("API Response:", response)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


NameError: name 'message' is not defined

Exception in thread Exception in thread Thread-24 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/httpcore/_exceptions.py", line 10, in map_exceptions
Thread-22 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/httpcore/_exceptions.py", line 10, in map_exceptions
    yield
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/httpcore/_backends/sync.py", line 168, in start_tls
    yield
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/httpcore/_backends/sync.py", line 168, in start_tls
    raise exc
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/httpcore/_backends/sync.py", line 163, in start_tls
    raise exc
  File "/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/pytho